# Tutorial: Extracting Information from Text Using OpenAI's API - format

This tutorial demonstrates how to use OpenAI's API to extract information from a text input. We will go through the steps of setting up the OpenAI client, making requests to the API, and processing the responses. The example involves summarizing a Danish news article and extracting structured information from it.

## Installation and Imports
First, we need to install the required library and import necessary modules.


In [ ]:
!pip install openai -q

In [ ]:
# Import required libraries
from openai import OpenAI
from google.colab import userdata
import json
from pydantic import BaseModel, Field
import textwrap

Setting Up the OpenAI Client

We will set up the OpenAI client using a custom API key and base URL.

In [ ]:
# Setup OpenAI client with custom API key and base URL
TOGETHER_API_KEY = userdata.get('TOGETHER_API_KEY')

### Summarizing Text

We will call the language model to summarize a given Danish text into a single sentence.

In [ ]:
# Create client
client = OpenAI(
    base_url="https://api.together.xyz/v1",
    api_key=TOGETHER_API_KEY
)

In [ ]:
text = """
The patient presented with complaints of persistent itching and redness on their hands, which they had noticed over the past week. They reported that the symptoms began shortly after they started using a new hand soap. The patient described the itching as intense and the redness as progressively worsening, with some areas of slight swelling.

During the consultation, the doctor conducted a thorough examination of the patient’s hands. The skin appeared erythematous, particularly on the dorsum of both hands. There were no signs of infection, such as pus or excessive warmth, but the localized erythema and slight edema were evident. The doctor noted that the distribution of the rash was consistent with contact dermatitis, likely caused by an irritant or allergen in the new soap.

The patient was asked about any other new products they had been using, to rule out other potential causes. No other new products or changes in routine were reported.

Given the history and clinical presentation, the doctor diagnosed the patient with contact dermatitis. The primary treatment recommendation was to discontinue the use of the suspected soap immediately. The doctor prescribed a topical corticosteroid cream to reduce inflammation and itching, advising the patient to apply it sparingly to the affected areas twice daily.

The doctor also provided general skin care advice to prevent further irritation: avoid hot water when washing hands, use a mild, fragrance-free soap, and apply a moisturizer regularly to maintain the skin barrier.

The patient was advised to monitor the symptoms and return for a follow-up appointment in two weeks to assess the effectiveness of the treatment. The doctor also mentioned that if the symptoms did not improve significantly or if they worsened, an allergy test might be considered to identify any specific allergens.

From the doctor’s perspective, the interaction was straightforward. The diagnosis was clear based on the patient’s history and symptoms. The prescribed treatment was expected to alleviate the symptoms, and the patient was receptive to the advice given. The doctor felt confident that with the recommended changes and medication, the patient’s condition would improve, but remained open to further investigation if necessary.

This detailed report captures both the patient’s experience and the doctor’s perspective, providing a comprehensive view of the consultation and the management plan.
"""

In [ ]:
# Call the LLM with the JSON schema
chat_completion = client.chat.completions.create(
    model="mistralai/Mixtral-8x7B-Instruct-v0.1",
    #model="meta-llama/Llama-3-70b-chat-hf",

    messages=[
        {
            "role": "system",
            "content": "You are a helpful clinical assistant. You only output what you have been asked for. No disclaimers, explanations, notes etc.",
        },
        {
            "role": "user",
            "content": "Create a 1 sentence summary of the following: " + text ,
        },
    ],
)

output = chat_completion.choices[0].message.content

In [ ]:
print(output)

## Creating a User Object and extracting structured info
We will define a schema for a user and call the API to create a user object based on this schema.

In [ ]:
# Define the schema for the output.
class User(BaseModel):
    name: str = Field(description="user name")
    address: str = Field(description="address")

In [ ]:
# Call the LLM with the JSON schema
chat_completion = client.chat.completions.create(
    model="mistralai/Mixtral-8x7B-Instruct-v0.1",
    response_format={"type": "json_object", "schema": User.model_json_schema()},
    messages=[
        {
            "role": "system",
            "content": "You are a helpful assistant that answers in JSON.",
        },
        {
            "role": "user",
            "content": "Create a user named Alice, who lives in 42, Wonderland Avenue.",
        },
    ],
)

created_user = json.loads(chat_completion.choices[0].message.content)
print(json.dumps(created_user, indent=2))

## Extracting Case Details
We will define a schema for case details and extract relevant information from the given text.

In [ ]:
class GPNoteDetails(BaseModel):
    date: str = Field(description="Date of the consultation")
    diagnosis: str = Field(description="Diagnosis given during the consultation")
    consultation_type: str = Field(description="Type of consultation (e.g., in-person, telephone, mail)")
    prescriptions: list[str] = Field(description="Medications or treatments prescribed during the consultation")
    notes: str = Field(description="Additional detailed notes from the consultation")

In [ ]:
# Call the LLM with the JSON schema
chat_completion = client.chat.completions.create(
    model="mistralai/Mixtral-8x7B-Instruct-v0.1",
    response_format={"type": "json_object", "schema": GPNoteDetails.model_json_schema()},
    messages=[
        {
            "role": "system",
            "content": "You are a helpful clinical assistant that answers in JSON.",
        },
        {
            "role": "user",
            "content": "Extract consulation information form following.: " + text,
        },
    ],
)

created_user = json.loads(chat_completion.choices[0].message.content)
print(json.dumps(created_user, indent=2))

In [ ]:
json_schema = """{'properties': {'date': {'description': 'Date of the consultation',
   'title': 'Date',
   'type': 'string'},
  'diagnosis': {'description': 'Diagnosis given during the consultation',
   'title': 'Diagnosis',
   'type': 'string'},
  'consultation_type': {'description': 'Type of consultation (e.g., in-person, telephone, mail)',
   'title': 'Consultation Type',
   'type': 'string'},
  'prescriptions': {'description': 'Medications or treatments prescribed during the consultation',
   'items': {'type': 'string'},
   'title': 'Prescriptions',
   'type': 'array'},
  'notes': {'description': 'Additional detailed notes from the consultation',
   'title': 'Notes',
   'type': 'string'}},
 'required': ['date',
  'diagnosis',
  'consultation_type',
  'prescriptions',
  'notes'],
 'title': 'GPNoteDetails',
 'type': 'object'}"""

In [ ]:
# Call the LLM with the JSON schema
chat_completion = client.chat.completions.create(
    model="meta-llama/Llama-3-70b-chat-hf",
    #response_format={"type": "json_object", "schema": CaseDetails.model_json_schema()},
    messages=[
        {
            "role": "system",
            "content": "You are a helpful clinical assistant that answers in JSON. No notes, no disclaimers, just follow JSON.",
        },
        {
            "role": "user",
            "content": "Extract from following: " + text + "Use the following JSON: " + json_schema,
        },
    ],
)



In [ ]:
created_note = json.loads(chat_completion.choices[0].message.content)
print(json.dumps(created_note, ensure_ascii=False, indent=2))


## Automation

In [ ]:
records_history = [
    {
        "date": "2024-05-10",
        "notes": "Patient reports pain in the right ear. Examination reveals redness and swelling in the ear canal. Prescribed antibiotic ear drops and advised to avoid water exposure."
    },
    {
        "date": "2024-06-01",
        "notes": "Patient complains of sore throat and cough. No fever. Throat appears red, and nasal congestion is present. Advised rest, fluids, and over-the-counter decongestants. Follow-up if symptoms worsen."
    },
    {
        "date": "2024-06-24",
        "notes": "Hemorrhoid. No bleeding. Prescribed cream. Follow-up to assess progress."
    },
    {
        "date": "2024-07-15",
        "notes": "Hemorrhoid has become a bit smaller. Has been using cream for 3 weeks. No bleeding. At inspection, a rather large hemorrhoid is seen at 4 o'clock. On rectal examination, a small fatty lump is felt in the same position. Plan to let Nyholm take a look. Continue with cream."
    },
    {
        "date": "2024-07-30",
        "notes": "Patient reports itching and redness on the hands after using a new soap. Examination shows localized erythema and slight swelling. Advised to discontinue use of the soap and prescribed topical corticosteroid."
    },
    {
        "date": "2024-08-10",
        "notes": "Patient called reporting headache, facial pain, and nasal discharge. Symptoms suggest sinusitis. Recommended steam inhalation and prescribed antibiotics over the phone. Follow-up in one week if no improvement."
    },
    {
        "date": "2024-08-20",
        "notes": "Patient emailed about persistent sneezing, runny nose, and itchy eyes, likely due to seasonal allergies. Recommended over-the-counter antihistamines and nasal spray. Advised to avoid known allergens and keep windows closed during high pollen days."
    }
]

In [ ]:
json_schema = """
  'diagnosis': {'description': 'Diagnosis given during the consultation',
   'title': 'Diagnosis',
   'type': 'string'},
  'consultation_type': {'description': 'Type of consultation (e.g., in-person, telephone, mail)',
   'title': 'Consultation Type',
   'type': 'string'},
  'prescriptions': {'description': 'Medications or treatments prescribed during the consultation',
   'items': {'type': 'string'},
   'title': 'Prescriptions',
   'type': 'array'},
  'notes': {'description': 'Additional detailed notes from the consultation',
   'title': 'Notes',
   'type': 'string'}},
 'required': ['diagnosis',
  'consultation_type',
  'prescriptions',
  'notes'],
 'title': 'GPNoteDetails',
 'type': 'object'}"""

In [ ]:
structured_records = []

for note in records_history:
  date = note["date"]
  notes = note["notes"]


  chat_completion = client.chat.completions.create(
      model="meta-llama/Llama-3-70b-chat-hf",
      #response_format={"type": "json_object", "schema": CaseDetails.model_json_schema()},
      messages=[
          {
              "role": "system",
              "content": "You are a helpful clinical assistant that answers in JSON. No notes, no disclaimers, just follow JSON.",
          },
          {
              "role": "user",
              "content": "Extract from following: " + notes + "Use the following JSON: " + json_schema,
          },
      ],
  )
  structrured_output = json.loads(chat_completion.choices[0].message.content)
  structrured_output['date'] = date
  structured_records.append(structrured_output)



In [ ]:
structured_records

In [ ]:
# Call the LLM with the JSON schema
chat_completion = client.chat.completions.create(
    model="mistralai/Mixtral-8x7B-Instruct-v0.1",
    #model="meta-llama/Llama-3-70b-chat-hf",

    messages=[
        {
            "role": "system",
            "content": "You are a helpful clinical assistant. You only output what you have been asked for. No disclaimers, explanations, notes etc.",
        },
        {
            "role": "user",
            "content": "Create a 2 sentence summary with the most import information for the doctor to be informed about the patients status: " + str(structured_records) ,
        },
    ],
)

output = chat_completion.choices[0].message.content

In [ ]:
wrapped_text = textwrap.fill(output, width=50)
print(wrapped_text)